# Working with the new Yr weather API

In [57]:
# some useful imports
import requests
import json
import xmltodict
import xml.etree.ElementTree as ET

In [58]:
lat = 53.86893
lon = 10.68729
alt = 162



In [37]:
class API_LocationForecast(object):

    """Class to use the API of api.met.no"""

    base_url = 'https://api.met.no/weatherapi/locationforecast/2.0/classic?'
    forecast_link = 'locationforecast'

    def __init__(self, lat, lon, alt):
        """
        :param double lat: latitude coordinate
        :param double lon: longitude coordinate
        """
        self.coordinates = dict(lat=lat, lon=lon, alt=alt)
        self.url = self.get_url()

    def get_url(self):
        """Return the url of API service"""
        return '{base_url}{location_name}'.format(
            base_url=self.base_url,
            location_name=self.location_name,
        )

In [59]:
# Create locationforecast url for API request

base_url = 'https://api.met.no/weatherapi/locationforecast/2.0/compact?'
api_url = '{base_url}lat={lat}&lon={lon}&altitude={alt}'.format(
    base_url=base_url,
    lat=lat,
    lon=lon,
    alt=alt
)

print(api_url)

https://api.met.no/weatherapi/locationforecast/2.0/compact?lat=53.86893&lon=10.68729&altitude=162


In [60]:
# Send request to API

sitename = 'https://jkl453.github.io/garden-123/'
headers = {'User-Agent': sitename}

response = requests.get(api_url, headers=headers)


print(response.status_code)
                                                                                                                                                                                                                                                             


200


In [61]:
#print(response.text)

response_json = json.loads(response.text)
print(json.dumps(response_json, indent=4))


{
    "type": "Feature",
    "geometry": {
        "type": "Point",
        "coordinates": [
            10.6873,
            53.8689,
            162
        ]
    },
    "properties": {
        "meta": {
            "updated_at": "2022-04-24T13:43:07Z",
            "units": {
                "air_pressure_at_sea_level": "hPa",
                "air_temperature": "celsius",
                "cloud_area_fraction": "%",
                "precipitation_amount": "mm",
                "relative_humidity": "%",
                "wind_from_direction": "degrees",
                "wind_speed": "m/s"
            }
        },
        "timeseries": [
            {
                "time": "2022-04-24T13:00:00Z",
                "data": {
                    "instant": {
                        "details": {
                            "air_pressure_at_sea_level": 1008.6,
                            "air_temperature": 8.8,
                            "cloud_area_fraction": 79.7,
                        